**This notebook is to test the computational time by CNN-driven advection scheme.**

This notebook perform loading the velocity field, implementing the machine-learned scheme, and wallclock time measurement.

In [1]:
## Import packages
#import Pkg
#Pkg.add("Plots")
using Plots
#Pkg.add("BenchmarkTools")
using BenchmarkTools
#Pkg.add("Flux")
using Flux
#Pkg.add("CSV")
using CSV
#Pkg.add("DelimitedFiles")
using DelimitedFiles
#Pkg.add("Statistics")
using Statistics
#Pkg.add("BSON")
using BSON: @save
using BSON: @load
#Pkg.add("StatsBase")
using StatsBase
#Pkg.add("Random")
using Random

In [2]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 3 stencil (default) ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim_total, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    s1_bc = zeros(xdim_total+2)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 3))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[i] for i in 1:xdim_total], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2]), xdim_total, 3)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [9]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 5 stencil ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    s1_bc = zeros(xdim_total+4)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 5))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[1]], [su[i] for i in 1:xdim_total], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4]), xdim_total, 5)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [20]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 9 stencil ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    #s1_bc = zeros(xdim_total+4)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 9))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
            [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8]), xdim_total, 9)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [2]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 17 stencil ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    #s1_bc = zeros(xdim_total+4)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 17))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
            [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], 
            [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8],
                [s1_bc[i] for i in 10:xdim_total+9],
                [s1_bc[i] for i in 11:xdim_total+10],
                [s1_bc[i] for i in 12:xdim_total+11],
                [s1_bc[i] for i in 13:xdim_total+12],
                [s1_bc[i] for i in 14:xdim_total+13],
                [s1_bc[i] for i in 15:xdim_total+14],
                [s1_bc[i] for i in 16:xdim_total+15],
                [s1_bc[i] for i in 17:xdim_total+16]), xdim_total, 17)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

pgm_ml (generic function with 1 method)

In [ ]:
## Integrating through the whole timesteps using ML based advection solver ##
## Programming advection scheme with 31 stencil ##
function pgm_ml(x, u1, model, xdim_total, nstep_total, dx, dt)
    
    ## Initialize
    history_2x_learned = zeros(Float32, xdim, 1, nstep)
    history_2x_learned[:,:,1] = x[:,1,1,1]
    s1_input = x[:,:,:,1]
    #s1_scale = s1_input
    #s1_bc = zeros(xdim_total+4)
    
    ## Integrate
    for n in 1:nstep_total-1
        # learned solver
        #coeff_estimated = model(hcat(s1_input, u1[:,:,:,n]))
        coeff_estimated = reshape(model(hcat(s1_input, u1[:,:,:,n])), (xdim_total, 2, 31))
        su = s1_input
        s1_bc = vcat( [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], 
                [su[1]], [su[1]], [su[1]], [su[1]], [su[1]], [su[i] for i in 1:xdim_total], 
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]],
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]], [su[xdim_total]],
                [su[xdim_total]], [su[xdim_total]], [su[xdim_total]])
        s1_scale = reshape(hcat([s1_bc[i] for i in 1:xdim_total],
                [s1_bc[i] for i in 2:xdim_total+1],
                [s1_bc[i] for i in 3:xdim_total+2],
                [s1_bc[i] for i in 4:xdim_total+3],
                [s1_bc[i] for i in 5:xdim_total+4],
                [s1_bc[i] for i in 6:xdim_total+5],
                [s1_bc[i] for i in 7:xdim_total+6],
                [s1_bc[i] for i in 8:xdim_total+7],
                [s1_bc[i] for i in 9:xdim_total+8],
                [s1_bc[i] for i in 10:xdim_total+9],
                [s1_bc[i] for i in 11:xdim_total+10],
                [s1_bc[i] for i in 12:xdim_total+11],
                [s1_bc[i] for i in 13:xdim_total+12],
                [s1_bc[i] for i in 14:xdim_total+13],
                [s1_bc[i] for i in 15:xdim_total+14],
                [s1_bc[i] for i in 16:xdim_total+15],
                [s1_bc[i] for i in 17:xdim_total+16],
                [s1_bc[i] for i in 18:xdim_total+17],
                [s1_bc[i] for i in 19:xdim_total+18],
                [s1_bc[i] for i in 20:xdim_total+19],
                [s1_bc[i] for i in 21:xdim_total+20],
                [s1_bc[i] for i in 22:xdim_total+21],
                [s1_bc[i] for i in 23:xdim_total+22],
                [s1_bc[i] for i in 24:xdim_total+23],
                [s1_bc[i] for i in 25:xdim_total+24],
                [s1_bc[i] for i in 26:xdim_total+25],
                [s1_bc[i] for i in 27:xdim_total+26],
                [s1_bc[i] for i in 28:xdim_total+27],
                [s1_bc[i] for i in 29:xdim_total+28],
                [s1_bc[i] for i in 30:xdim_total+29],
                [s1_bc[i] for i in 31:xdim_total+30]), xdim_total, 31)
        s2_2x = reshape(s1_input, xdim_total) + 100*dt/dx*sum(coeff_estimated[:,1,:].*s1_scale, dims=2) 
                + 10000*(dt*dt)/(dx*dx)*sum(coeff_estimated[:,2,:].*s1_scale, dims=2)
        
        history_2x_learned[:,1,n+1] = s2_2x
        s1_input = reshape(s2_2x, (xdim_total,1,1))
    end
    
    return history_2x_learned
end

In [9]:
## This cell performs the data loading, specifying the dimension, and model loading.
vel_GEOS_Array = readdlm( "Training_dataset/Vel_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
scalar_GEOS_Array = readdlm("Training_dataset/VL_GEOS_Jan_2019_NASA_GMAO_10_U_16x_64x.csv", ',', Float32);
xdim = size(vel_GEOS_Array, 1);
nstep = size(vel_GEOS_Array, 2);

Random.seed!(1)
history = (scalar_GEOS_Array*Float32(1e7) + 4e-5*rand(Float32, xdim, nstep), vel_GEOS_Array/15);

## Call variables
coeff_estimated = zeros(Float32, xdim,2)
#input = coarse_grained_s_2x[:,:]
s2_2x = convert(Array{Float64}, zeros(xdim))
history_2x_learned = zeros(xdim, 1, nstep);

input_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
u_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)
target_NN_integrate = zeros(Float32, xdim, 1, 1, nstep-1)

input_NN_integrate[:,1,1,:] = history[1][:,1:nstep-1]
u_NN_integrate[:,1,1,:] = history[2][:,1:nstep-1]
target_NN_integrate[:,1,1,:] = history[1][:,2:nstep]

@load "Train_outputs/16x64t/CNN_DLR_56EPOCHS_16X64X_VL_GEOS_JAN_2019_NASA_GMAO.bson_MODEL" model
ps = Flux.params(model);

In [10]:
## This code evaluate the computational time in a single run.
@time A = 1e-7*pgm_ml(input_NN_integrate, u_NN_integrate, model, xdim, nstep, Float32(27034.3*16), 300*64);

  0.002172 seconds (11.14 k allocations: 1.551 MiB)


In [12]:
## This code evaluate the computational time in a more desirable way.
## Evaluate multiple times to take the average.
## If the running time is large enough to neglect the initialization and garbage collection time,
## @benchmark only evaluates the time once.
@benchmark A = 1e-7*pgm_ml(input_NN_integrate, u_NN_integrate, model, xdim, nstep, Float32(27034.3*16), 300*64)

BenchmarkTools.Trial: 2435 samples with 1 evaluation.
 Range (min … max):  1.772 ms … 20.746 ms  ┊ GC (min … max): 0.00% … 88.12%
 Time  (median):     1.884 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.049 ms ±  1.212 ms  ┊ GC (mean ± σ):  6.37% ±  9.13%

  █                                                           
  █▃▄▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▂
  1.77 ms        Histogram: frequency by time        12.1 ms <

 Memory estimate: 1.55 MiB, allocs estimate: 11139.